In [27]:
import numpy as np
import numba as nb
from numba.typed import List
import pandas as pd
import itertools
from operator import itemgetter
import gc

from scipy.stats import norm, lognorm, truncnorm
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, deque
from sys import getsizeof
from sedgen.initialization import SedGen
from sedgen import initialization as ini
from sedgen import general as gen
from sedgen import preprocessing
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time 
# before executing the Python code typed.
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
def optimise_learnrate(minerals, par_rock_volume, modal_mineralogy, means, stds):
    learnrate = 100
    counter = 0
    while counter != 10:
        model = SedGen(minerals, par_rock_volume, modal_mineralogy, means, stds, learning_rate = learnrate, discretization_init=False, timed = True)
        modelled_volume = np.sum(model.simulated_volume)
        print(par_rock_volume)
        print(modelled_volume)
        if modelled_volume < ((par_rock_volume*0.0001)+par_rock_volume) and modelled_volume> (par_rock_volume-(par_rock_volume*0.0001)): 
            break
        else:
            learnrate *= 10
        print(counter, learnrate)
        counter += 1
    pluton_learnrate = learnrate * par_rock_volume
    return (pluton_learnrate)
    
            

In [29]:
minerals = ["Q", "P", "K", "B", "O", "A"]

In [30]:
modal_mineralogy = np.array([[0.16331256, 0.46387757, 0.12683457, 0.05758711, 0.01078412, 0.17760407],   # AZ
                             [0.27453254, 0.40124204, 0.24993859, 0.02509231, 0.00454988, 0.04464465],   # CA-EW
                             [0.30591989, 0.38159713, 0.26209888, 0.0188256 , 0.00799247, 0.02356603],   # CA-NS
                             [0.19781274, 0.41766034, 0.16121575, 0.06478527, 0.00894429, 0.14958161],   # GR
                             [0.25983144, 0.46392506, 0.15827298, 0.04370113, 0.00689158, 0.06737782],   # MT
                             [0.22419158, 0.48490082, 0.05641885, 0.11571367, 0.        , 0.11877509]])
print(modal_mineralogy[0])

[0.16331256 0.46387757 0.12683457 0.05758711 0.01078412 0.17760407]


In [31]:
csds_AZ_means = np.array([0.120, 0.174, 0.112, 0.155, 0.066, 0.079])
csds_AZ_stds = np.array([0.359, 0.356, 0.455, 0.399, 0.328, 0.452])

csds_CA_NS_means = np.array([0.309, 0.330, 0.244, 0.223, 0.120, 0.122])
csds_CA_NS_stds = np.array([0.823, 0.683, 0.817, 0.819, 0.554, 0.782])

csds_CA_EW_means = np.array([0.250, 0.350, 0.254, 0.265, 0.199, 0.250])
csds_CA_EW_stds = np.array([0.850, 0.700, 0.650, 0.780, 0.900, 0.765])

csds_GR_means = np.array([0.150, 0.150, 0.100, 0.200, 0.100, 0.090])
csds_GR_stds = np.array([0.400, 0.450, 0.500, 0.450, 0.300, 0.280])

csds_MT_means = np.array([0.200, 0.214, 0.320, 0.265, 0.189, 0.111])
csds_MT_stds = np.array([0.500, 0.490, 0.485, 0.651, 0.355, 0.455])

csds_WA_means = np.array([0.199, 0.252, 0.298, 0.350, 0.178, 0.156])
csds_WA_stds = np.array([0.422, 0.546, 0.635, 0.743, 0.546, 0.413])


weightedmeanAZ = np.sum(modal_mineralogy[0]*csds_AZ_means)
weightedmeanCA_EW = np.sum(modal_mineralogy[1]*csds_AZ_means)
weightedmeanCA_NS = np.sum(modal_mineralogy[2]*csds_CA_NS_means)
weightedmeanGR = np.sum(modal_mineralogy[3]*csds_AZ_means)
weightedmeanMT = np.sum(modal_mineralogy[4]*csds_AZ_means)
weightedmeanWA = np.sum(modal_mineralogy[5]*csds_AZ_means)

In [32]:
yaxis = np.array([weightedmeanAZ,weightedmeanCA_EW,weightedmeanCA_NS,weightedmeanGR,weightedmeanMT,weightedmeanWA])
learnvalueAZ = optimise_learnrate(minerals, 1e9, modal_mineralogy[0], csds_AZ_means, csds_AZ_stds)
learnvalueCA_EW = optimise_learnrate(minerals, 1e9, modal_mineralogy[1], csds_CA_EW_means, csds_CA_EW_stds)
learnvalueCA_NS = optimise_learnrate(minerals, 1e9, modal_mineralogy[2], csds_CA_NS_means, csds_CA_EW_stds)
learnvalueGR = optimise_learnrate(minerals, 1e9, modal_mineralogy[3], csds_GR_means, csds_GR_stds)
learnvalueMT = optimise_learnrate(minerals, 1e9, modal_mineralogy[4], csds_MT_means, csds_MT_stds)
learnvalueWA = optimise_learnrate(minerals, 1e9, modal_mineralogy[5], csds_WA_means, csds_WA_stds)
xaxis = np.array([learnvalueAZ, learnvalueCA_EW, learnvalueCA_NS, learnvalueGR, learnvalueMT, learnvalueMT, learnvalueWA])

---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A| Done in 36.6530 seconds
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[1 4 1 ... 0 0 0]
 Done in 86.5571 seconds
Counting interfaces...  Done in 0.9114 seconds
Correcting interface arrays for consistency...
too much Q 10
too few P -1
too few K -3
too few B -3
too few O -1
too few A -2
Initializing crystal size array... |Q|P|K|B|O|A| Done in 5.9953 seconds
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...

---SedGen model initialization finished succesfully---
1000000000.0
1018032533.1023808
0 1000
---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A| Done in 37.6204 seconds
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[1 3 0 ... 0 0 0]
 Done in

ValueError: need at least one array to concatenate

---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A| Done in 24.2307 seconds
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[2 0 1 ... 1 0 0]
 Done in 64.2205 seconds
Counting interfaces...  Done in 0.7182 seconds
Correcting interface arrays for consistency...
too few Q -7
too much P 1
too much K 1
too much B 2
too much O 1
too much A 2
Initializing crystal size array... |Q|P|K|B|O|A| Done in 4.4068 seconds
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...

---SedGen model initialization finished succesfully---
1000000000.0
5896667275.989795
0 1000
---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A| Done in 4.8614 seconds
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[1 2 3 ... 0 0 0]
 Done in 12

10000000000000.0